In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

category_dropdown = None
file_dropdown = None
process_button = None
next_button = None
counter_label = None
output = widgets.Output()

image_dict = {}
file_index = {}


def setup_interface(on_process, base_path="../data/images"):
    global category_dropdown, file_dropdown, process_button, next_button, counter_label, image_dict, file_index

    categories = ["clean", "medium", "heavy"]
    image_dict = {
        cat: sorted(
            [f for f in os.listdir(os.path.join(base_path, cat)) if f.endswith('.png')]
        )
        for cat in categories
    }
    file_index = {cat: 0 for cat in categories}

    category_dropdown = widgets.Dropdown(options=categories, description="Категория:")
    file_dropdown = widgets.Dropdown(description="Файл:")
    process_button = widgets.Button(description="Обработать модель", button_style="success")
    next_button = widgets.Button(description="Следующая", button_style="info")
    counter_label = widgets.Label()

    def update_file_options(*args):
        cat = category_dropdown.value
        files = image_dict[cat]
        idx = file_index[cat]
        file_dropdown.options = files
        file_dropdown.value = files[idx]
        update_counter()

    def update_counter():
        cat = category_dropdown.value
        files = image_dict[cat]
        current_file = file_dropdown.value
        if current_file in files:
            file_index[cat] = files.index(current_file)
            current = file_index[cat] + 1
            total = len(files)
            counter_label.value = f"{current}/{total}"
        else:
            counter_label.value = "?"

    def on_next_click(_):
        cat = category_dropdown.value
        files = image_dict[cat]
        file_index[cat] = (file_index[cat] + 1) % len(files)
        file_dropdown.value = files[file_index[cat]]
        update_counter()
        on_process()

    def on_file_change(*args):
        update_counter()

    process_button.on_click(lambda _: on_process())
    next_button.on_click(on_next_click)
    category_dropdown.observe(update_file_options, names='value')
    file_dropdown.observe(on_file_change, names='value')

    update_file_options()

    ui = widgets.VBox([
        category_dropdown,
        widgets.HBox([file_dropdown, counter_label]),
        widgets.HBox([process_button, next_button]),
        output
    ])

    display(ui)


def on_button_click(b=None):
    category = category_dropdown.value
    filename = file_dropdown.value
    with output:
        clear_output(wait=True)
        plot_lidar(category, filename)


setup_interface(on_process=on_button_click)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull
import imageio.v3 as iio
import json
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from truck_bed_detection import detect_truck_bed


def plot_lidar(category, filename, base_path="../data/images", gt_path="../data/ground-truth"):
    img_path = os.path.join(base_path, category, filename)
    img = iio.imread(img_path)

    h, w = img.shape
    angle_min, angle_max = -30, 30
    angles = np.radians(np.linspace(angle_min, angle_max, w))
    times = np.arange(h) * 0.02  # 50 Hz

    angle_grid, time_grid = np.meshgrid(angles, times)

    r = img / 1000.0
    r[r > 20] = 0

    X = r * np.sin(angle_grid)
    Y = time_grid
    Z = -r * np.cos(angle_grid)

    data = [go.Scatter3d(
        x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
        mode='markers',
        marker=dict(size=1, color=Z.flatten(), colorscale='Viridis', opacity=0.8),
        name="LiDAR"
    )]
    (A, B, C, D), inlier_points = detect_truck_bed(X, Y, Z)

    inlier_mask = np.isnan(inlier_points[:, 0]) == False
    inlier_X = inlier_points[inlier_mask, 0]
    inlier_Y = inlier_points[inlier_mask, 1]
    inlier_Z = inlier_points[inlier_mask, 2]

    x_plane, y_plane = np.meshgrid(np.linspace(np.min(inlier_X), np.max(inlier_X), 50),
                                   np.linspace(np.min(inlier_Y), np.max(inlier_Y), 50))
    z_plane = -(A * x_plane + B * y_plane + D) / C

    data.append(go.Surface(
        x=x_plane, y=y_plane, z=z_plane,
        colorscale='Magma', opacity=0.4,
        name="Plane",
        colorbar=dict(
            x=-0.15,
            title='Глубина дна кузова',
            title_side='right',
            len=0.5,
        ),
    ))

    points = np.column_stack((inlier_X, inlier_Y, inlier_Z))
    hull = ConvexHull(points[:, :2])

    hull_x = points[hull.vertices, 0]
    hull_y = points[hull.vertices, 1]
    hull_z = points[hull.vertices, 2]

    data.append(go.Scatter3d(
        x=hull_x, y=hull_y, z=hull_z,
        mode='lines+markers',
        line=dict(color='red', width=4),
        marker=dict(size=5, color='red'),
        name="Hull"
    ))

    json_name = filename.replace('.png', '.jpg.json')
    json_path = os.path.join(gt_path, category, json_name)
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            gt = json.load(f)

        if gt["objects"]:
            poly_px = np.array(gt["objects"][0]["data"])

            px_x = np.array([pt[0] for pt in poly_px])
            px_y = np.array([pt[1] for pt in poly_px])

            gt_angles = np.radians(angle_min + (px_x / (w - 1)) * (angle_max - angle_min))
            gt_times = px_y * 0.02

            gt_r = img[px_y.astype(int), px_x.astype(int)] / 1000.0
            gt_r[gt_r > 20] = 0

            gt_X = gt_r * np.sin(gt_angles)
            gt_Y = gt_times
            gt_Z = -gt_r * np.cos(gt_angles)

            data.append(go.Scatter3d(
                x=gt_X, y=gt_Y, z=gt_Z,
                mode='lines+markers',
                line=dict(color='orange', width=4),
                marker=dict(size=3, color='orange'),
                name="Ground Truth"
            ))

    fig = go.Figure(data=data)
    fig.update_layout(
        scene=dict(
            xaxis_title="X (угол)",
            yaxis_title="Время (сек)",
            zaxis_title="Глубина (м)",
            aspectmode='cube'
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        title=f"{category}/{filename}"
    )
    fig.show()